In [87]:
import torch
import os

import d2l.torch as d2l

In [88]:
#@save
class TokenEmbedding:
    """GloVe嵌入"""

    def __init__(self, embedding_model_name, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(embedding_model_name,
                                                                  embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_model_name, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        # data_dir = d2l.download_extract(embedding_name)
        # GloVe网站：https://nlp.stanford.edu/projects/glove/
        # fastText网站：https://fasttext.cc/
        data_path = r'D://DataModel//' + embedding_model_name + '//' + embedding_name
        with open(data_path, 'r', encoding='utf-8') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # 跳过标题信息，例如fastText中的首行
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

In [89]:
# glove_6b50d = TokenEmbedding('glove', 'glove.6b.50d.txt')
glove_6b50d = TokenEmbedding('fasttext', 'wiki-news-300d-1M.vec')

In [90]:
len(glove_6b50d)

999995

In [91]:
idx = glove_6b50d.token_to_idx['jejune']
print(idx)
token = glove_6b50d.idx_to_token[301540]
print(token)

178586
Eskisehir


KNN函数
通余弦相似性来查找语义相似的词：
$$
\text { similarity }=\cos (\theta)=\frac{A \cdot B}{\|A\|\|B\|}=\frac{\sum_{i=1}^{n} A_{i} \times B_{i}}{\sqrt{\sum_{i=1}^{n}\left(A_{i}\right)^{2}} \times \sqrt{\sum_{i=1}^{n}\left(B_{i}\right)^{2}}}
$$

In [92]:
def knn(W, x, k):
    # 增加1e-9以获得数值稳定性
    cos = torch.mv(W, x.reshape(-1, )) / (
            torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
            torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

In [93]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # 排除输入词
        print(f'{embed.idx_to_token[int(i)]}：cosine相似度={float(c):.3f}')


In [94]:
get_similar_tokens('chip', 3, glove_6b50d)

chips：cosine相似度=0.829
microchip：cosine相似度=0.649
silicon：cosine相似度=0.643


In [95]:
get_similar_tokens('baby', 3, glove_6b50d)

babies：cosine相似度=0.803
newborn：cosine相似度=0.763
Baby：cosine相似度=0.722


In [96]:
get_similar_tokens('beautiful', 3, glove_6b50d)

gorgeous：cosine相似度=0.843
lovely：cosine相似度=0.828
charming：cosine相似度=0.781


词类比
通过word_a、word_b、word_c来类比第四个词

$$
\operatorname{vec_d} = \operatorname{vec_a} + \operatorname{vec_b} - \operatorname{vec_c}
$$

In [97]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])]  # 删除未知词

In [98]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'china'

In [99]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'son'

In [100]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'

In [101]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'